In [ ]:
import os
import json
import numpy as np
import itertools


input_folder = "output_data_kmeans"
output_folder = "output_select_path_greedy"
os.makedirs(output_folder, exist_ok=True)

# tính ma trận thời gian di chuyển 
def compute_travel_time(cluster_heads, positions, v_AUV=10.0):
    n = len(cluster_heads)
    travel_time = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                dist = np.linalg.norm(positions[cluster_heads[i]] - positions[cluster_heads[j]])
                travel_time[i][j] = dist / v_AUV
    return travel_time

# chọn đường đi 
def path_selection(cluster_heads, travel_time):
    n = len(cluster_heads)
    best_time = float('inf')
    best_path = None
    for path in itertools.permutations(range(n)):
        total_time = sum(travel_time[path[i-1]][path[i]] for i in range(1, n))
        if total_time < best_time:
            best_time = total_time
            best_path = path
    return best_path, best_time


for filename in os.listdir(input_folder):
    if not filename.endswith(".json"):
        continue

    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, f"result_{filename}")

    print(f"\nĐang xử lý {filename}")

    with open(input_path, "r", encoding="utf-8") as f:
        nodes_data = json.load(f)

    # Trích danh sách cluster head và tọa độ tâm cụm
    cluster_heads = []
    positions = {}
    for cid, info in nodes_data.items():
        ch = info["cluster_head"]
        cluster_heads.append(ch)
        positions[ch] = np.array(info["center"])

    # Tính ma trận thời gian di chuyển
    travel_time = compute_travel_time(cluster_heads, positions)

    # Tìm đường đi tối ưu
    best_path, best_time = path_selection(cluster_heads, travel_time)

    # Xuất kết quả
    result = {
        "input_file": filename,
        "cluster_heads": cluster_heads,
        "best_path": [cluster_heads[i] for i in best_path],
        "total_time": round(float(best_time), 4)
    }

    with open(output_path, "w", encoding="utf-8") as f_out:
        json.dump(result, f_out, ensure_ascii=False, indent=4)

    print(f"Kết quả của {output_path}:")
    print(f"Tổng thời gian di chuyển nhỏ nhất: {best_time:.2f} s")

print("\nDone")



Đang xử lý nodes_20.json
Kết quả của output_select_path_greedy\result_nodes_20.json:
Tổng thời gian di chuyển nhỏ nhất: 30.05 s

Done
